## What is Semantic Kernel?

Semantic Kernel is a model-agnostic SDK that empowers developers to build, orchestrate, and deploy AI agents and multi-agent systems. Whether you're building a simple chatbot or a complex multi-agent workflow, Semantic Kernel provides the tools you need with enterprise-grade reliability and flexibility.

## System Requirements

- **Python**: 3.10+
- **.NET**: .NET 8.0+ 
- **Java**: JDK 17+
- **OS Support**: Windows, macOS, Linux

## Key Features

- **Model Flexibility**: Connect to any LLM with built-in support for [OpenAI](https://platform.openai.com/docs/introduction), [Azure OpenAI](https://azure.microsoft.com/en-us/products/ai-services/openai-service), [Hugging Face](https://huggingface.co/), [NVidia](https://www.nvidia.com/en-us/ai-data-science/products/nim-microservices/) and more
- **Agent Framework**: Build modular AI agents with access to tools/plugins, memory, and planning capabilities
- **Multi-Agent Systems**: Orchestrate complex workflows with collaborating specialist agents
- **Plugin Ecosystem**: Extend with native code functions, prompt templates, OpenAPI specs, or Model Context Protocol (MCP)
- **Vector DB Support**: Seamless integration with [Azure AI Search](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search), [Elasticsearch](https://www.elastic.co/), [Chroma](https://docs.trychroma.com/getting-started), and more
- **Multimodal Support**: Process text, vision, and audio inputs
- **Local Deployment**: Run with [Ollama](https://ollama.com/), [LMStudio](https://lmstudio.ai/), or [ONNX](https://onnx.ai/)
- **Process Framework**: Model complex business processes with a structured workflow approach
- **Enterprise Ready**: Built for observability, security, and stable APIs


## 💡 Initialize our Project

In [1]:
import asyncio
import logging
import os
import sys
from dotenv import load_dotenv
import importlib
from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)



import Plugins.KnowledgeBasePlugin as kb_plugin # Import the KnowledgeBasePlugin
import Plugins.LightsPlugin as lights_plugin # Import the LightsPlugin
importlib.reload(kb_plugin) 
importlib.reload(lights_plugin) 

KnowledgeBasePlugin = kb_plugin.KnowledgeBasePlugin
LightsPlugin = lights_plugin.LightsPlugin


# Set up the Endpoint and API Key for Azure OpenAI and Azure Search
load_dotenv(dotenv_path="variables.env")

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
gpt_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
azure_search_key = os.getenv("AZURE_SEARCH_KEY")

# Print the loaded environment variables for debugging


## 💡 Initialize Kernel and add AzureChatCompletion

This code block demonstrates how to initialize a Semantic Kernel, which serves as the central component for managing AI services and plugins, and configure it to use Azure OpenAI's chat completion service.



In [2]:
# Initialize Kernel
kernel = Kernel()


# Set up Azure OpenAI completion service
chat_completion = AzureChatCompletion(
    deployment_name=gpt_deployment,
    api_key=azure_openai_api_key,  
    base_url=azure_openai_endpoint,
)

# Add the service to the kernel
kernel.add_service(chat_completion)

# Logging setup
setup_logging()
logging.getLogger("kernel").setLevel(logging.DEBUG)


## 💡 Add Plugins to the Kernel to invoke the functions

In [3]:
# Add the Lights plugin 
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)

# Remove previous plugin instance if needed
if "KnowledgeBase" in kernel.plugins:
    del kernel.plugins["KnowledgeBase"]


# Add the KnowledgeBase plugin 
kernel.add_plugin(
    KnowledgeBasePlugin(
        endpoint=azure_search_endpoint, 
        index_name="default", 
        api_key=azure_search_key
        ),
    plugin_name="KnowledgeBase",
)

for plugin_name, plugin in kernel.plugins.items():
    print(f"\n🧩 Plugin: {plugin_name}")
    for func in plugin:
        print(f"  └─ {func.name}: {func.description}")




🧩 Plugin: Lights
  └─ change_state: Changes the state of the light
  └─ get_lights: Gets a list of lights and their current state

🧩 Plugin: KnowledgeBase
  └─ ask_with_context: Answers a user question using top 5 relevant documents from Azure Search.
  └─ search_docs: Retrieve top 3 documents from the knowledge base that match a query. Includes filename and a 500-word preview of each result.


# 🧠 Planner
Semantic Kernel leverages function calling–a native feature of most LLMs–to provide planning. With function calling, LLMs can request (or call) a particular function to satisfy a user's request. Semantic Kernel then marshals the request to the appropriate function in your codebase and returns the results back to the LLM so the AI agent can generate a final response.

Function choice behaviors are bits of configuration that allow a developer to configure:

- 📣 **Which functions are advertised to AI models.**  
- 🧭 **How the models should choose them for invocation.**  
- ⚙️ **How Semantic Kernel might invoke those functions.**

---

## 📚 Available Function Choice Behaviors

As of today, the function choice behaviors are represented by three class methods of the `FunctionChoiceBehavior` class:

### 🔄 `Auto`
💡 *Allows the AI model to choose from zero or more function(s) from the provided function(s) for invocation.*

### ✅ `Required`
📌 *Forces the AI model to choose one or more function(s) from the provided function(s) for invocation.*

### 🚫 `NoneInvoke`
⛔ *Instructs the AI model not to choose any function(s).*


In [ ]:
# Set up execution settings
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto() 

In [ ]:
# Create conversation history
history = ChatHistory()


async def chat_with_assistant(user_input: str):
    history.add_user_message(user_input)
    print("User >", user_input)
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    print("Assistant >", result)
    
    history.add_message(result)

## 💡 Test it!

In [32]:
await chat_with_assistant("Can you show me the current state of all the lights?")
await chat_with_assistant("Can you turn on the table lamp light?")
await chat_with_assistant("What about now? Are all the lights on?")

User > Can you show me the current state of all the lights?
Assistant > Here is the current state of all the lights:

1. **Table Lamp**: On
2. **Porch Light**: Off
3. **Chandelier**: On
User > Can you turn on the table lamp light?
Assistant > The Table Lamp is now turned on.
User > What about now? Are all the lights on?
Assistant > Currently, the state of the lights is as follows:

1. **Table Lamp**: On
2. **Porch Light**: Off
3. **Chandelier**: On

Not all the lights are on; the Porch Light is off.


In [33]:
await chat_with_assistant("Use the knowledge base to explain what is a Residential Contract of Sale.")


User > Use the knowledge base to explain what is a Residential Contract of Sale.
Assistant > A Residential Contract of Sale is a legally binding agreement for the sale and purchase of improved single-family residential real estate. It establishes the terms and conditions under which the property will be sold, including:

- **Parties Involved**: It clearly outlines the seller and buyer's names and addresses.
- **Property Details**: It specifies the location and legal description of the property being sold.
- **Purchase Price**: The total price of the property, along with details regarding any deposits or payment schedules.
- **Contingencies**: Conditions that must be satisfied for the sale to proceed, such as financing approval or inspection results.
- **Timeframes**: Important deadlines, including the date of the contract, settlement date, and any time-sensitive actions that must occur.
- **Responsibilities**: It details the obligations of both parties regarding inspections, repairs, a

In [28]:
await chat_with_assistant("Show me documents about seller disclosures.")


Assistant > Here are some documents related to seller disclosures:

1. **[Manmaya-handwritten-single-family-home-contract.pdf]**:
   - This document is a contract for the sale of a property located at 2345 Oak Ave, Los Angeles, CA. It includes provisions for seller disclosures, the responsibilities of both parties, and specifies the need for earnest money and inspection rights. The closing date is set for on or before October 1, 2024.

2. **[Oscar-Ward-Real-Estate-Contract.pdf]**:
   - This contract outlines a residential sale in Maryland and includes detailed disclosures concerning lead poisoning prevention and other potential environmental hazards. It also includes an addendum for various disclosures required by law, emphasizing the seller's responsibilities related to health and safety standards.

3. **[Eric-handwritten-new-construction-contract.pdf]**:
   - This document relates to a contract for a new construction property sale in San Antonio, TX. It details seller obligations, su

## 💡 Try to create your own Plugin 


Below is a list of easy, API-free Semantic Kernel plugins attendees can build during the workshop:

---

### ✅ WeatherPlugin
**Functionality:**
- `get_weather(city)`: Returns temperature, humidity, and condition.
- `get_temperature(city)`: Returns only the temperature.
- `get_humidity(city)`: Returns only the humidity.

---

### ✅ TodoPlugin
**Functionality:**
- `add_task(task)`: Adds a task to the list.
- `list_tasks()`: Lists all current tasks.
- `remove_task(index)`: Removes a task by its index.

---

### ✅ MusicPlugin
**Functionality:**
- `add_song(title)`: Adds a song to the playlist.
- `now_playing()`: Shows the currently playing song.
- `skip_song()`: Skips the current song.

---

### ✅ BookTrackerPlugin
**Functionality:**
- `add_book(title)`: Adds a book to the reading list.
- `mark_read(title)`: Marks a book as read.
- `list_books()`: Lists all books with their statuses.

---

### ✅ HabitPlugin
**Functionality:**
- `check_in()`: Logs a habit check-in and updates streak.



## Resources to help you:

- https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/?pivots=programming-language-python
- https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/adding-native-plugins?pivots=programming-language-python